### Imports

In [457]:
from backtesting.test import EURUSD, GOOG



### Process parquet

In [458]:
import pandas as pd
import pyarrow.parquet as pq

DATA_PATH='/home/darrendube/Documents/spatialedge-hackathon-H1/v1/' 
FROM_DATE='20100101'
TO_DATE='20221231'
SYMBOL = 'EURCAD'
TIMEFRAME = 'H1'

def read_and_process_parquet(data_path, from_date, to_date, symbol, timeframe):
    partition = ['symbol','timeframe', 'date', 'date']
    operator = ['=', '=', '>=', '<=']
    params = [symbol, timeframe, from_date, to_date]
            
    dataset = pq.ParquetDataset(data_path, filters=list(zip(partition, operator, params)), use_legacy_dataset=True)
    table = dataset.read()
    df = table.to_pandas()

    df['date'] = df['date'].astype(str)
    df['time'] = df['time'].astype(str)

    df['datetime'] = df['date'] + ' ' + df['time']
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y%m%d %H:%M:%S')
    df.set_index('datetime', inplace=True)

    df.drop(['time', 'symbol', 'timeframe', 'date'], axis=1, inplace=True)
    df = df.sort_values('datetime')
    df.fillna(method='ffill', inplace=True)

    return df 

df = read_and_process_parquet(DATA_PATH, FROM_DATE, TO_DATE, SYMBOL, TIMEFRAME)

/tmp/ipykernel_81488/3887751971.py:15: FutureWarning: Passing 'use_legacy_dataset=True' to get the legacy behaviour is deprecated as of pyarrow 11.0.0, and the legacy implementation will be removed in a future version.
  dataset = pq.ParquetDataset(data_path, filters=list(zip(partition, operator, params)), use_legacy_dataset=True)


### Create helper functions

In [459]:
data = EURUSD.copy()

# Relative Strength Index (RSI)
close = data.Close.values


def RSI(values, period=168):
    delta = pd.Series(values).diff()
    gain = delta.where(delta>0.0)
    #gain.fillna(0, inplace=True) 
    loss = -delta.where(delta<0.0)
    #loss.fillna(0, inplace=True) #TODO: instead of replacing with 0, remove those vaues
    loss
    avg_gain = gain.rolling(period, min_periods=1).mean() # average gain over 7 day period (168hr) 
    avg_gain.fillna(0, inplace=True)
    avg_loss = loss.rolling(period, min_periods=1).mean() # average loss over 7 day period (168hr)
    avg_loss.fillna(0.000000001, inplace=True)
    rs = (avg_gain/avg_loss)
    rsi = 100-(100/(1+rs))
   
    return rsi

def SMA(values, n):
    
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

def Stochastic_Oscillator(values, period=168):
    '''Calculates stochastic oscillator'''
    time_period = period
    data = pd.Series(values)
    if len(values) < time_period:
        time_period = len(values)
    lowest_close_in_period = data.rolling(time_period, min_periods=1).min()
    highest_close_in_period = data.rolling(time_period, min_periods=1).max()
    so = ((data-lowest_close_in_period)/(highest_close_in_period-lowest_close_in_period))*100
    return so


In [460]:
Stochastic_Oscillator(data.Close.values)


0              NaN
1       100.000000
2         0.000000
3        14.705882
4         0.000000
           ...    
4995      7.670773
4996      3.247480
4997      0.447928
4998      3.471445
4999      0.000000
Length: 5000, dtype: float64

### Strategy based on Relative Strength Index

In [461]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

    
class RulesBasedStrategy(Strategy):
    # TODO: add rules based
    # The SMA periods
    n1 = 10
    n2 = 20

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.rsi = self.I(RSI, self.data.Close, 23) # Calculate 1 day rsi
        self.stochastic_oscillator = self.I(Stochastic_Oscillator, self.data.Close, 168)

    def next(self):
        # If all indicators say you should buy, buy
        if self.rsi < 30 or self.stochastic_oscillator< 20 or crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
        
        # If all indicators say you should sell, sell
        elif self.rsi > 70 or self.stochastic_oscillator > 80 or crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
        

        

### Backtest

In [462]:
df.columns = ['Open','High','Low','Close', 'Volume']
bt = Backtest(df, RulesBasedStrategy, cash=1000000, commission=0.000)
stats = bt.run()
stats
#bt.plot()

Start                     2010-01-03 22:00:00
End                       2022-12-30 21:00:00
Duration                   4743 days 23:00:00
Exposure Time [%]                   99.973407
Equity Final [$]                1006352.12422
Equity Peak [$]                 1336970.34243
Return [%]                           0.635212
Buy & Hold Return [%]                -3.28905
Return (Ann.) [%]                    0.040323
Volatility (Ann.) [%]                  8.2382
Sharpe Ratio                         0.004895
Sortino Ratio                        0.006769
Calmar Ratio                         0.001129
Max. Drawdown [%]                   -35.71378
Avg. Drawdown [%]                   -0.894222
Max. Drawdown Duration     4337 days 08:00:00
Avg. Drawdown Duration       39 days 12:00:00
# Trades                                 4390
Win Rate [%]                        37.790433
Best Trade [%]                       5.827798
Worst Trade [%]                     -4.120228
Avg. Trade [%]                    